## BERT Sentiment Analysis

In [11]:
import pandas as pd
import spacy
from transformers import pipeline
import re

In [12]:
data = pd.read_csv("data/refugee_coca_foranalysis.csv")

In [13]:
try:
    nlp = spacy.load("en")
except OSError:
    nlp = spacy.load("en_core_web_sm")

def sent_tokenize(word_list, model=nlp):
    doc = model(word_list)
    sentences = [sent.string.strip() for sent in doc.sents]
    return sentences

In [ ]:
data['sentences'] = data['text'].apply(sent_tokenize)

In [ ]:
#5-year periods
def get_period(year, startyr, endyr, n=5):
    period_start = []
    for i in range(startyr, endyr+1, n):
        period_start.append(i)
    for index, p in enumerate(period_start):
        if year >= p:
            period = index
            continue
        else:
            break
    return period  
    
data['period'] = data['year'].apply(lambda x: get_period(x, 1991, 2015, n=5) if x>=1991 else 0)

In [ ]:
# Write a function that let us conveniently label ideology: Left, Neutral, Right
media_ideology = {}

def label_ideology(media_title, ideology, media_ideology = media_ideology):
    media_ideology[media_title] = ideology
    
label_ideology('Money', 'Neutral')
label_ideology('MotherEarth', 'Neutral')
label_ideology('MotherJones', 'Left')
label_ideology('AmHeritage', 'Neutral')
label_ideology('AmSpect', 'Right')
label_ideology('Forbes', 'Right')
label_ideology('NatlReview', 'Right')
label_ideology('Newsweek', 'Left')
label_ideology('ScienceNews', 'Neutral')
label_ideology('Smithsonian', 'Neutral')
label_ideology('USNWR', 'Left')
label_ideology('WashMonth', 'Left')
label_ideology('ChangingTimes', 'Right')
label_ideology('HistoryToday', 'Neutral')
label_ideology('Omni', 'Neutral')
label_ideology('Wilderness', 'Neutral')
label_ideology('TIME', 'Left')
label_ideology('NatlParks', 'Neutral')
label_ideology('AmerArtist', 'Neutral')
label_ideology('RollingStone', 'Left')
label_ideology('Americas', 'Neutral')
label_ideology('SportsIll', 'Neutral')
label_ideology('Ms', 'Left')
label_ideology('PopScience', 'Neutral')
label_ideology('Futurist', 'Neutral')
label_ideology('HarpersMag', 'Left')
label_ideology('Fortune', 'Right')
label_ideology('USAToday', 'Left')
label_ideology('America', 'Left')
label_ideology('ChristCentury', 'Right')
label_ideology('People', 'Left')
label_ideology('Jet', 'Left')
label_ideology('Aging', 'Neutral')
label_ideology('Horticulture', 'Neutral')
label_ideology('NewRepublic', 'Left')
label_ideology('Conservation', 'Left')
label_ideology('NaturalHist', 'Neutral')
label_ideology('Atlantic', 'Left')
label_ideology('Inc.', 'Neutral')
label_ideology('ChildrenToday', 'Neutral')
label_ideology('Ebony', 'Left')
label_ideology('ConsumResrch', 'Neutral')
label_ideology('SatEvenPost', 'Neutral')
label_ideology('ChristToday', 'Right')
label_ideology('Backpacker', 'Neutral')
label_ideology('AmericanCraft', 'Neutral')
label_ideology('ArtAmerica', 'Neutral')
label_ideology('SportingNews', 'Neutral')
label_ideology('MensHealth', 'Neutral')
label_ideology('Antiques', 'Neutral')
label_ideology('Parenting', 'Neutral')
label_ideology('Essence', 'Neutral')
label_ideology('Environmental', 'Neutral')
label_ideology('USCatholic', 'Right')
label_ideology('MilitaryHist', 'Neutral')
label_ideology('PsychToday', 'Neutral')
label_ideology('Cosmopolitan', 'Left')
label_ideology('Redbook', 'Neutral')
label_ideology('Bazaar', 'Left')
label_ideology('ChildDigest', 'Neutral')
label_ideology('Bicycling', 'Neutral')
label_ideology('Shape', 'Neutral')
label_ideology('NatGeog', 'Neutral')
label_ideology('Entertainment', 'Neutral')
label_ideology('Astronomy', 'Neutral')
label_ideology('TownCountry', 'Neutral')
label_ideology('TotalHealth', 'Neutral')
label_ideology('Esquire', 'Left')
label_ideology('FieldStream', 'Neutral')
label_ideology('TechReview', 'Neutral')
label_ideology('CountryLiving', 'Neutral')
label_ideology('VegTimes', 'Neutral')
label_ideology('SouthernLiv', 'Neutral')
label_ideology('Skiing', 'Neutral')
label_ideology('ConsumRep', 'Neutral')
label_ideology('Sunset', 'Neutral')
label_ideology('HarpersBazaar', 'Neutral')
label_ideology('AmericanSpectator', 'Right')
label_ideology('GoodHousekeeping', 'Neutral')
label_ideology('PopMech', 'Neutral')
label_ideology('MHQTheQuarterly', 'Neutral')
label_ideology('TodaysParent', 'Neutral')
label_ideology('NationalGeographic', 'Neutral')
label_ideology('EEnvironmental', 'Neutral')
label_ideology('ParentingEarly', 'Neutral')
label_ideology('ABC', 'Left')
label_ideology('CNN', 'Left')
label_ideology('PBS', 'Left')
label_ideology('CBS', 'Left')
label_ideology('Ind', 'Left')
label_ideology('NPR', 'Left')
label_ideology('NBC', 'Left')
label_ideology('Fox', 'Right')
label_ideology('MSNBC', 'Left')
label_ideology('NYTimes', 'Left')
label_ideology('CSMonitor', 'Neutral')
label_ideology('AssocPress', 'Neutral')
label_ideology('WashPost', 'Left')
label_ideology('SanFranChron', 'Left')
label_ideology('Atlanta', 'Left') #Atlanta Journal Constitution
label_ideology('Houston', 'Left') #Houston Chronicle
label_ideology('Chicago', 'Left') #Chicago Sun-Times
label_ideology('Denver', 'Left') #Denver Post
label_ideology('GolfMag', 'Neutral')
label_ideology('NewStatesman', 'Left')
label_ideology('Austin', 'Left') #Austin American Statesman
label_ideology('STLouis', 'Left') #St Louis Post_Dispatch
label_ideology('Pittsburgh', 'Right') #Pittsburgh Post-Gazette
label_ideology('OrangeCR', 'Right') #Orange County Register

#add political leaning label
def add_ideology(x, media_ideology=media_ideology):
    try_split = re.split("_|: | ",x)
    if len(try_split)>1:
        x = try_split[0]
    if x in media_ideology:
        return media_ideology[x]
    else:
        print('{} does not exists'.format(x))

data["ideology"] = data['source'].apply(add_ideology)

In [ ]:
data.head()

In [ ]:
# Allocate a pipeline for sentiment-analysis
nlp_sentiment = pipeline('sentiment-analysis')

In [ ]:
periods = data.period.unique()

sentences = {}
sentiment_over_period = {}
sentiment_scores = []
for i in ['Left', 'Right']:
    for p in periods:
        data_period = data[(data['period'] == p) & (data['ideology']==i)]
        for sent in data_period['sentences'].sum():
            if 'refugee' in sent: 
                if '{}_{}'.format(p,ideology) not in sentences:
                    sentences['{}_{}'.format(p,ideology)] = []
                sentences['{}_{}'.format(p,ideology)].append(sent)
                sentiment = nlp_sentiment(sent)
                polarity = sentiment[0]['label']
                score = sentiment[0]['score']

                if polarity=='NEGATIVE':
                    score = -score

                sentiment_scores.append(score)
        avg = sum(sentiment_scores) / len(sentiment_scores)
        sentiment_over_period['{}_{}'.format(p,ideology)] = avg

In [ ]:
sentences[1]

In [ ]:
sentiment_over_period

## BERT Text Generation

In [ ]:
from transformers import AutoModelWithLMHead, AutoTokenizer

In [ ]:
def textgen_results(sequence):
    refugees_textgen = {}
    tokenizer_gpt = AutoTokenizer.from_pretrained("gpt2")
    model_gpt = AutoModelWithLMHead.from_pretrained("gpt2")

    input = tokenizer_gpt.encode(sequence, return_tensors="pt")
    generated = model_gpt.generate(input, max_length=50)
    resulting_string = tokenizer_gpt.decode(generated.tolist()[0])
    refugees_textgen['gpt'] = resulting_string

    for ideology in ['left','right']:
        tokenizer = AutoTokenizer.from_pretrained("bertresults/output_gpt_{}".format(ideology))
        model = AutoModelWithLMHead.from_pretrained("bertresults/output_gpt_{}".format(ideology))

        input = tokenizer.encode(sequence, return_tensors="pt")
        generated = model.generate(input, max_length=50, bos_token_id=1, pad_token_id=1, eos_token_ids=1)
        resulting_string = tokenizer.decode(generated.tolist()[0])
        refugees_textgen['all_{}'.format(ideology)] = resulting_string
        
        for period in range(0,5):
            tokenizer = AutoTokenizer.from_pretrained("bertresults/output_gpt_period{}_{}".format(period, ideology))
            model = AutoModelWithLMHead.from_pretrained("bertresults/output_gpt_period{}_{}".format(period, ideology))

            input = tokenizer.encode(sequence, return_tensors="pt")
            generated = model.generate(input, max_length=50, bos_token_id=1, pad_token_id=1, eos_token_ids=1)
            resulting_string = tokenizer.decode(generated.tolist()[0])

            refugees_textgen['period{}_{}'.format(period, ideology)] = resulting_string
    return refugees_textgen

In [ ]:
refugees_textgen = textgen_results("Refugees are")
refugees_textgen

In [ ]:
textgen_results("Israeli refugees are")

In [ ]:
textgen_results("Syrian refugees are")